In [33]:
!cd "Java Projects\GraphsImplementations" && javac -d bin src/uniandes/algorithms/graphs/*.java && java -cp bin uniandes.algorithms.graphs.GraphImplementations minCost Dijkstra "data\graph.txt"


Minimum Cost Matrix using Dijkstra: 

      |    0    1    2    3    4
---------------------------------
    0 |    0   90   80   92  119
    1 |   15    0   69   48   84
    2 |   65   51    0   12   39
    3 |   62   48   75    0   36
    4 |   26   12   39   33    0

Execution time in microseconds: 1274


In [21]:
!cd "Java Projects\GraphsImplementations" && javac -d bin src/uniandes/algorithms/graphs/*.java && java -cp bin uniandes.algorithms.graphs.GraphImplementations minCost BellmanFord "data\graph.txt"



Minimum Cost Matrix using BellmanFord: 

      |    0    1    2    3    4
---------------------------------
    0 |    0   90   80   92  119
    1 |   15    0   69   48   84
    2 |   65   51    0   12   39
    3 |   62   48   75    0   36
    4 |   26   12   39   33    0

Execution time in microseconds: 326


In [22]:
!cd "Java Projects\GraphsImplementations" && javac -d bin src/uniandes/algorithms/graphs/*.java && java -cp bin uniandes.algorithms.graphs.GraphImplementations minCost FloydWarshall "data\graph.txt"



Minimum Cost Matrix using FloydWarshall: 

      |    0    1    2    3    4
---------------------------------
    0 |  105   90   80   92  119
    1 |   15   96   69   48   84
    2 |   65   51   78   12   39
    3 |   62   48   75   69   36
    4 |   26   12   39   33   69

Execution time in microseconds: 79
